In [10]:
%load_ext rpy2.ipython 

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [11]:
%%R 
w_max=5 # max wild-type plasmids
m_max=5 # max mutant plasmids, equal to m_lethal-1

nt_init=matrix(0,nrow=w_max+1,ncol=m_max+1)
w_init=5
m_init=0
nt_init[w_init+1,m_init+1]=3.12e9
rownames(nt_init)=paste0('w',0:w_max)
colnames(nt_init)=paste0('m',0:m_max)

In [12]:
%%R 
cell_division_old <- function(nt){
  nt_div=matrix(0,nrow=w_max+1,ncol=m_max+1)
  rownames(nt_div)=paste0('w',0:w_max)
  colnames(nt_div)=paste0('m',0:m_max)
  for(a in 0:w_max){ # loop over the entire population
    for(b in 0:m_max){ 
      if((a+b)>=1){ # require a+b>0
        
        for(jpk in 0:(a+b)){ # loop over the total number of plasmids in first cell
          p_jpk=dbinom(jpk,a+b,0.5) # binomial distribution
          
          for(j in max(0,jpk-b):min(a,jpk)){
            new=p_jpk*dhyper(j,a,b,jpk)*nt[a+1,b+1]
            nt_div[j+1,jpk-j+1]=nt_div[j+1,jpk-j+1]+new # cell 1
            nt_div[a-j+1,b-(jpk-j)+1]=nt_div[a-j+1,b-(jpk-j)+1]+new # cell 2
          }
        }
      }
    }
  }
  return (nt_div)
}

cell_division <- function(nt){
  nt_div=nt*0
  for(a in 0:w_max){ # loop over the entire population
    for(b in 0:m_max){ 
      
      p_nt=dbinom(0:(a+b),a+b,0.5)*nt[a+1,b+1] # binomial distribution * number of cells
      for(jpk in 0:(a+b)){ # loop over the total number of plasmids in first cell
        j_all=max(0,jpk-b):min(a,jpk) # number of wild-type plasmids in first cell
        new_all=p_nt[jpk+1]*dhyper(j_all,a,b,jpk) # number of cells
        # row=j_all+1
        # column=jpk-j_all+1
        # element=(column-1)*(w_max+1)+(row)
        nt_div[(jpk-j_all)*(w_max+1)+j_all+1]=nt_div[(jpk-j_all)*(w_max+1)+j_all+1]+new_all # cell 1
        # row=a-j_all+1
        # column=b-(jpk-j_all)+1
        # element=(column-1)*(w_max+1)+(row)
        nt_div[(b-jpk+j_all)*(w_max+1)+a-j_all+1]=nt_div[(b-jpk+j_all)*(w_max+1)+a-j_all+1]+new_all # cell 1
      }
    }
  }
  return (nt_div)
}


In [13]:
%%R 
#u=0.01 # wt -> mutant rate
#r=2 # wt plasmid growth rate
#R=3 # mutant plasmid growth rate 

plasmid_replication_mutation_old <- function(nt_div,u,r,R){
    nt_growth=matrix(0,nrow=w_max+1,ncol=m_max+1)
    rownames(nt_growth)=paste0('w',0:w_max)
    colnames(nt_growth)=paste0('m',0:m_max)
    
    for(a in 0:w_max){ # loop over the entire population
      for(b in 0:m_max){ 
        a_plus1=floor(a*r)
        b_plus1=floor(b*R)
        
        new_wt=a_plus1-a
        w_to_m=0:new_wt
          
        a_plus2=a_plus1-w_to_m
        b_plus2=b_plus1+w_to_m
    
        a_plus2=ifelse (a_plus2<=w_max,a_plus2,w_max)
        b_plus2=ifelse (b_plus2<=m_max,b_plus2,-5) # if too many mutant plasmids, the cell dies
                                                   # mutant coord = a random negative number -5
        for (i in 0:new_wt) {
          if(b_plus2[i+1]>=0)
            nt_growth[a_plus2[i+1]+1,b_plus2[i+1]+1]=
                nt_growth[a_plus2[i+1]+1,b_plus2[i+1]+1] +
                nt_div[a+1,b+1]*dbinom(i,new_wt,u)
        }
      }       
    }
    return (nt_growth)   
}

plasmid_replication_mutation <- function(nt_div,u,r,R){
  nt_growth=nt_div*0
  
  for(a in 0:w_max){ # loop over the entire population
    a_plus1=floor(a*r)
    new_wt=a_plus1-a
    w_to_m=0:new_wt
    a_plus2=a_plus1-w_to_m
    a_plus2=ifelse(a_plus2<=w_max,a_plus2,w_max) ##### this is not in the model
    
    p_w_to_m=dbinom(w_to_m,new_wt,u)
    
    for(b in 0:m_max){ 
      b_plus1=floor(b*R)
      b_plus2=b_plus1+w_to_m
      b_plus2=ifelse(b_plus2<=m_max,b_plus2,-5) # if too many mutant plasmids, the cell dies
      
      new=nt_div[a+1,b+1]*p_w_to_m
      for (i in w_to_m+1){
        if(b_plus2[i]>=0)
          nt_growth[a_plus2[i]+1,b_plus2[i]+1]=nt_growth[a_plus2[i]+1,b_plus2[i]+1]+new[i]
      }
      
    }       
  }
  return (nt_growth)   
}


In [14]:
%%R 
#S=0.8
#m_optimal=12 # NOTE: need m_optimal<=m_max (since m_max+1=m_lethal)

cell_selection_old <-function(nt_growth, S, m_optimal){
    nt_selection=matrix(0,nrow=w_max+1,ncol=m_max+1)
    rownames(nt_selection)=paste0('w',0:w_max)
    colnames(nt_selection)=paste0('m',0:m_max)
    
    for(a in 0:w_max){ # loop over the entire population
      for(b in 0:m_max){
          if (b==0){
              nt_selection[a+1,b+1]=nt_growth[a+1,b+1]*S
          } else if (0<b & b<=m_optimal){
              nt_selection[a+1,b+1]=nt_growth[a+1,b+1]*(S+(1-S)*b/m_optimal)
          } else if (m_optimal<b){
              nt_selection[a+1,b+1]=nt_growth[a+1,b+1]*((m_max-b)/(m_max-m_optimal))
          } 
          if (b+a==0){
              nt_selection[a+1,b+1] = 0
          }
      }
    }
    return (nt_selection)
}

cell_selection <-function(nt_growth, S, m_optimal){
  nt_selection=nt_growth*0
  
  nt_selection[,1]=nt_growth[,1]*S # b=0
  nt_selection[1,1]=0 # a+b=0
  nt_selection[,1:m_optimal+1]=t(t(nt_growth[,1:m_optimal+1])*(S+(1-S)*(1:m_optimal)/m_optimal))
  if(m_optimal<m_max)
    nt_selection[,(m_optimal+1):m_max+1]=t(t(nt_growth[,(m_optimal+1):m_max+1])*((m_max-(m_optimal+1):m_max)/(m_max-m_optimal)))
  
  return (nt_selection)
}


In [15]:
%%R 
fcell_2_fplasmid_old <- function(nt_selection){
    wt_plasmid=0
    mutant_plasmid=0
    
    for(a in 0:w_max){ # loop over the entire population
      for(b in 0:m_max){
        wt_plasmid=wt_plasmid+nt_selection[a+1,b+1]*a
        mutant_plasmid=mutant_plasmid+nt_selection[a+1,b+1]*b
      }
    }
    
    return (wt_plasmid/(wt_plasmid+mutant_plasmid))    
}

fcell_2_fplasmid <- function(nt_selection){
  
  wt_plasmid=sum(nt_selection*(0:w_max))
  mutant_plasmid=sum(t(nt_selection)*(0:m_max))
  
  return (wt_plasmid/(wt_plasmid+mutant_plasmid))    
}

In [16]:
%%R 
generation=318
mutant_plasmid_freq=rep(-1,generation)
nt=nt_init

emperical_data=read.delim("F3_data1.tab", header=TRUE)
emperical_generations=emperical_data$generations

loss_old<-function(theta){
  u=theta[1]
  r=theta[2]
  R=theta[3]
  S=theta[4]
  m_optimal=round(theta[5]) # m_optimal needs to be integer!
  
  if(u<0 | u>1 | r<1 | R<1 | S<0 | S>1 | m_optimal<=0 | m_optimal>m_max) {
    return (100000000)
  }
  
  for (i in 1:generation){
    nt_div=cell_division_old(nt)
    nt_growth=plasmid_replication_mutation_old(nt_div,u,r,R)
    nt_selection=cell_selection_old(nt_growth, S, m_optimal)
    mutant_plasmid_freq[i]=fcell_2_fplasmid_old(nt_selection)
    nt=nt_selection
  } 
  
  diff=mutant_plasmid_freq[emperical_generations]-emperical_data$frequency
  return (sum(diff^2))
  
} 

loss<-function(theta){
    u=theta[1]
    r=theta[2]
    R=theta[3]
    S=theta[4]
    m_optimal=round(theta[5]) # m_optimal needs to be integer!
    
    if(u<=0 | u>=1 | r<1 | R<1 | S<0 | S>1 | m_optimal<=0 | m_optimal>m_max) {
        return(+Inf)
    }
    
    for (i in 1:generation){
        nt_div=cell_division(nt)
        nt_growth=plasmid_replication_mutation(nt_div,u,r,R)
        nt_selection=cell_selection(nt_growth, S, m_optimal)
        mutant_plasmid_freq[i]=fcell_2_fplasmid(nt_selection)
        nt=nt_selection
    } 

    diff=mutant_plasmid_freq[emperical_generations]-emperical_data$frequency
    return (sum(diff^2))
    
}

In [17]:
%%R 
theta0=c(0.01, 1.2, 1.8, 0.8, 3)
#res=optim (theta0,loss,control=c(maxit=1e6))
print (loss_old(theta0))
print (loss(theta0))

[1] 13.84734
[1] 13.84734


In [18]:
%%R 
# old code
ptm=proc.time()
res_old=optim(theta0,loss_old,control=c(maxit=1e6))
print(res_old)
proc.time()-ptm

$par
[1] 0.3257106 1.4392277 2.2889755 0.3166912 2.7775563

$value
[1] 0.0009712628

$counts
function gradient 
      36       NA 

$convergence
[1] 0

$message
NULL

   user  system elapsed 
 64.121   0.509  68.095 


In [19]:
%%R
# new code
ptm=proc.time()
res=optim(theta0,loss,control=c(maxit=1e6))
print(res)
proc.time()-ptm
print (proc.time()-ptm)

identical(res_old,res)

$par
[1] 0.3257106 1.4392277 2.2889755 0.3166912 2.7775563

$value
[1] 0.0009712628

$counts
function gradient 
      36       NA 

$convergence
[1] 0

$message
NULL

   user  system elapsed 
 33.805   0.279  35.103 
[1] TRUE
